$$\textbf{Machine Learning. Tarea 1: Métodos de descenso para Funciones de base radial}$$
$$\textit{Y. Sarahi García Gozález}$$

In [138]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image
import cv2

<font size=4 color='mediumslateblue'>  

$\textit{Librerías}$

In [139]:
print("Tarea realizada en MacOs. \nLas versiones de las librerías y de python utilizadas fueron:\n")
from platform import python_version
print("Python version:", python_version())
print("NumPy version:", np.__version__)
print("Pandas version:", pd.__version__)


Tarea realizada en MacOs. 
Las versiones de las librerías y de python utilizadas fueron:

Python version: 3.11.5
NumPy version: 1.23.5
Pandas version: 2.1.4


<font size=4 color='mediumslateblue'>  

$\textit{Prepocesamiento de la imagen}$

In [140]:
#leemos la imagen
Lena = Image.open("Lena_image.png")
#obtenemos la matriz
lena_matrix = np.array(Lena)
#normalizamos
lena_matrix=lena_matrix/np.max(lena_matrix)
#convertimos a vector
lena_vector = lena_matrix.flatten()

In [141]:
print(lena_matrix.shape) # (256, 256)

(256, 256)


In [142]:
print(lena_matrix) # los valores están entre o y 1

[[0.18333333 0.18333333 0.05833333 ... 0.40833333 0.40833333 0.01666667]
 [0.18333333 0.18333333 0.05833333 ... 0.43333333 0.40833333 0.01666667]
 [0.24166667 0.         0.00833333 ... 0.10833333 0.125      0.575     ]
 ...
 [0.25       0.15       0.21666667 ... 0.45       0.38333333 0.44166667]
 [0.25833333 0.15       0.15       ... 0.26666667 0.23333333 0.15833333]
 [0.20833333 0.14166667 0.15       ... 0.23333333 0.23333333 0.33333333]]


In [143]:
lena_vector.shape #256*256=65,536

(65536,)

In [144]:
lena_vector

array([0.18333333, 0.18333333, 0.05833333, ..., 0.23333333, 0.23333333,
       0.33333333])

Buscamos aproximar la imagen anterior pensandola como una función $f$ que a cada pixel $x=(x_1,x_2)$ le asigna un valor entre 0 y 1. Dicha aproximación se hará mediante una suma de funciones:

$$f(x) = \sum_{j=1}^J \alpha_j \phi(x; \theta_j) + \eta(x) $$ 

con $\eta(x)$ un residual

In [153]:
# Definición de laa funciones base y us gradientes
def base_gaussiana(x, theta, kappa):
    r2 = (x - theta)**2
    return np.exp(-kappa * r2)

def grad_gaussiana (x, theta, kappa):
    r2 = (x - theta)**2
    return kappa*(2*x - 2*theta)*(np.exp(-kappa*r2))

def base_multiquadrica(x, theta, kappa):
    r2 = (x - theta)**2
    return np.sqrt(r2+kappa)

def grad_multiquadrica(x, theta, kappa):
    r2 = (x - theta)**2
    resultado = -(x - theta) / np.sqrt(kappa + r2)
    return resultado


<font size=4 color='mediumslateblue'>  

$\textit{Parte 1}$

In [157]:
def genera_phi(theta,f_base,imagen_vector,kappa):
    '''

    '''
    N2=len(imagen_vector)
    J=len(theta)
    phi=np.zeros((N2,J))

    for j in range(J):
        phi[:,j]=np.array([f_base(i, theta[j],kappa) for i in range(N2)])

    return phi

In [158]:
J=20
theta = np.random.uniform(low=0, high=1, size = J)
kappa=0.1
Phi=genera_phi(theta,base_multiquadrica,lena_vector,kappa)



In [159]:
Phi.shape

(65536, 20)

In [160]:
theta

array([0.10788725, 0.55979066, 0.61579066, 0.82379337, 0.56378538,
       0.99481439, 0.12587708, 0.40380765, 0.41729809, 0.3624047 ,
       0.45889985, 0.17353278, 0.71407808, 0.22233647, 0.12530818,
       0.39249197, 0.51489843, 0.3670855 , 0.80902706, 0.20024711])

In [161]:
# Calcula la pseudoinversa de Moore-Penrose de phi
phi_pseudoinv = np.linalg.pinv(Phi)
# Resuelve el sistema utilizando la pseudoinversa de Moore-Penrose
alpha = np.dot(phi_pseudoinv, lena_vector )

In [162]:
aproximacion=np.dot(Phi,alpha)

In [163]:
# Convierte el vector a una matriz
matriz = np.reshape(lena_vector, (256, 256))
# Escala la matriz a valores entre 0 y 255 (0 corresponde a negro y 255 a blanco)
matriz = (matriz - np.min(matriz)) / (np.max(matriz) - np.min(matriz)) * 255
# Convierte la matriz a tipo de datos entero
matriz = matriz.astype(np.uint8)
# Crea la imagen a partir de la matriz
imagen = cv2.cvtColor(matriz,cv2.COLOR_GRAY2BGR)

In [164]:
cv2.imwrite("imagen_generada6.png", imagen)

True

In [155]:
np.shape(Phi)

(65536, 20)

<font size=4 color='mediumslateblue'>  

$\textit{Parte 2}$

In [ ]:
def grad_phi(phi,theta):
    N2,J=np.shape(phi)
    phi_grad=np.zeros((N2,J))

    for i in range(N2):
        for j in range(J):
            phi_grad[i][j]=(theta[j]-i)/phi[i][j]
    
    return(phi_grad)



def grad_funcion_costo(alpha,phi,x,f):
    grad = phi.T@(phi@x - f)

def genera_phi(theta,f_base,imagen_vector,kappa):
    '''

    '''
    N2=len(imagen_vector)
    J=len(theta)
    phi=np.zeros((N2,J))

    for j in range(J):
        phi[:,j]=np.array([f_base(x, theta[j],kappa) for x in imagen_vector])

    return phi

In [154]:
def gradient_descent():
    

SyntaxError: incomplete input (4220602832.py, line 2)